In [1]:
from ortools.sat.python import cp_model
from operator import add
import pandas as pd
import itertools
from common import Data

In [2]:
df = pd.read_csv("2024.csv")
df["season"] = "2024"
teams = list(df["home"].unique())
n = len(teams)
Data.get_nwsl_standings(df)

,team,wins,draws,losses,goals_for,goals_against,goals_diff,points
1,ORL,14,5,0,37.0,12.0,25.0,47
2,WAS,13,2,4,39.0,21.0,18.0,41
3,KC,11,5,3,43.0,28.0,15.0,38
4,NJY,11,4,4,23.0,17.0,6.0,37
5,NC,10,1,8,24.0,20.0,4.0,31
6,POR,8,3,8,29.0,27.0,2.0,27
7,BFC,8,0,11,24.0,31.0,-7.0,24
8,CHI,7,2,10,25.0,28.0,-3.0,23
9,LA,6,3,10,22.0,31.0,-9.0,21
10,RGN,5,5,9,23.0,32.0,-9.0,20


In [3]:
# Calculate points for all teams so far
def calc_initial_points(df):
    standings = Data.get_nwsl_standings(df).set_index("team").reindex(teams)
    return list(standings["points"])

In [4]:
# Create 2d array for how many games remain between each pair of teams
def calc_games_matrix(df):
    future_games = df[df["home_score"].isna()]
    g = [[0] * n for i in range(n)]
    for i in range(n):
        for j in range(n):
            if i < j:
                count = len(
                    future_games[
                        (
                            (future_games["home"] == teams[i])
                            & (future_games["away"] == teams[j])
                        )
                        | (
                            (future_games["away"] == teams[i])
                            & (future_games["home"] == teams[j])
                        )
                    ].index
                )
                g[i][j] = count
                g[j][i] = count
    return g

In [5]:
# For the provided team, calculates the lowest the team could finish assuming they
# lose all remaining games. If this number is 8 or higher than the team has
# clinched a playoff spot.
def calculate_lowest_finish(team, g, p0):
    # Creates the model.
    model = cp_model.CpModel()

    ####################################################
    # Create the variables.

    # Team we are checking to see if they clinched
    k = teams.index(team)
    # w[i][j] represents the number of wins of i over j
    w = [
        [model.new_int_var(0, 2, f"w_{i}_{j}") if i != j else None for j in range(n)]
        for i in range(n)
    ]
    # t[i][j] represents the number of ties between i and j
    t = [
        [model.new_int_var(0, 2, f"t_{i}_{j}") if i != j else None for j in range(n)]
        for i in range(n)
    ]
    # p[i] represents the number of points team i has
    p = [model.new_int_var(0, 3 * 26, f"p_{i}") for i in range(n)]
    # b[i] is 1 if team i has more points than team k
    b = [model.new_bool_var(f"b_{i}") for i in range(n)]

    ####################################################
    # Create the constraints.

    # Wins must sum up to the total games
    for i in range(n):
        for j in range(n):
            if i < j:
                model.add(w[i][j] + w[j][i] + t[i][j] == g[i][j])

    # Ties must be symmetric
    for i in range(n):
        for j in range(n):
            if i != j:
                model.add(t[i][j] == t[j][i])

    # Points calculation
    for i in range(n):
        if i == k:
            model.add(p[i] == p0[i])
        else:
            model.add(
                p[i]
                == p0[i]
                + 3 * sum(w[i][j] for j in filter(lambda x: x != i, range(n)))
                + 1 * sum(t[i][j] for j in filter(lambda x: x != i, range(n)))
            )

    # Calculating if a team is better than team k
    for i in range(n):
        model.add(p[i] >= p[k]).only_enforce_if(b[i])

    ####################################################
    # Set the optimization function.
    model.maximize(sum(b[i] for i in range(n)))

    # Creates a solver and solves the model.
    solver = cp_model.CpSolver()
    status = solver.solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        return sum(solver.value(b[i]) if i != k else 0 for i in range(n)) + 1
    else:
        return None

In [6]:
g = calc_games_matrix(df)
p0 = calc_initial_points(df)
calculate_lowest_finish("ORL", g, p0)

6

In [7]:
g = calc_games_matrix(df)
p0 = calc_initial_points(df)
calculate_lowest_finish("WAS", g, p0)

9

In [8]:
g = calc_games_matrix(df)
p0 = calc_initial_points(df)
calculate_lowest_finish("KC", g, p0)

11

In [9]:
# For the provided team, calculates the highest the team could finish assuming they
# win all remaining games. If this number is 8 or higher, the team hasn't been
# eliminated.
def calculate_highest_finish(team, g, p0):    
    # Creates the model.
    model = cp_model.CpModel()

    ####################################################
    # Create the variables.

    # Team we are checking to see if they clinched
    k = teams.index(team)
    # w[i][j] represents the number of wins of i over j
    w = [
        [model.new_int_var(0, 2, f"w_{i}_{j}") if i != j else None for j in range(n)]
        for i in range(n)
    ]
    # t[i][j] represents the number of ties between i and j
    t = [
        [model.new_int_var(0, 2, f"t_{i}_{j}") if i != j else None for j in range(n)]
        for i in range(n)
    ]
    # p[i] represents the number of points team i has
    p = [model.new_int_var(0, 3 * 26, f"p_{i}") for i in range(n)]
    # b[i] is 1 if team i has less points than team k
    b = [model.new_bool_var(f"b_{i}") for i in range(n)]

    ####################################################
    # Create the constraints.

    # Wins must sum up to the total games
    for i in range(n):
        for j in range(n):
            if i < j:
                model.add(w[i][j] + w[j][i] + t[i][j] == g[i][j])

    # Ties must be reflexive
    for i in range(n):
        for j in range(n):
            if i != j:
                model.add(t[i][j] == t[j][i])

    # Points calculation
    for i in range(n):
        if i == k:
            model.add(p[i] == p0[i] + 3 * sum(g[i][j] for j in range(n)))
        else:
            model.add(
                p[i]
                == p0[i]
                + 3 * sum(w[i][j] for j in filter(lambda x: x != i, range(n)))
                + 1 * sum(t[i][j] for j in filter(lambda x: x != i, range(n)))
            )

    # Calculating if a team is worse than team k
    for i in range(n):
        model.add(p[i] <= p[k]).only_enforce_if(b[i])

    ####################################################
    # Set the optimization function.
    model.maximize(sum(b[i] for i in range(n)))

    # Creates a solver and solves the model.
    solver = cp_model.CpSolver()
    status = solver.solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        return n - sum(solver.value(b[i]) if i != k else 0 for i in range(n))
    else:
        return None

In [10]:
g = calc_games_matrix(df)
p0 = calc_initial_points(df)
calculate_highest_finish("HOU", g, p0)

5

In [11]:
# Calculate all scenarios for the following week and whether a specific team
# has clinched a playoff spot

team = "WAS"

start = min(df.index[df["home_score"].isna()])
df_copy = df.copy()
df_copy.loc[start : start + (n / 2 - 1), "home_score"] = [0] * int(n / 2)
df_copy.loc[start : start + (n / 2 - 1), "away_score"] = [0] * int(n / 2)

g = calc_games_matrix(df_copy)
p0 = calc_initial_points(df)

def calculate_clinch(row):
    subset = df_copy.loc[start : start + (n / 2 - 1)]
    subset["home_score"] = list(row.str[0])
    subset["away_score"] = list(row.str[1])
    return (
        calculate_lowest_finish(
            team, g, list(map(add, p0, calc_initial_points(subset)))
        )
        <= 8
    )


outcomes = list([(0, 0), (1, 0), (0, 1)])
scenarios = pd.DataFrame(
    itertools.product(outcomes, repeat=int(n / 2))
)
scenarios["clinched"] = scenarios.apply(calculate_clinch, axis=1)
scenarios

,0,1,2,3,4,5,6,clinched
0,"(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)",True
1,"(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)","(1, 0)",True
2,"(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 1)",True
3,"(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)","(1, 0)","(0, 0)",True
4,"(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)","(1, 0)","(1, 0)",True
...,...,...,...,...,...,...,...,...
2182,"(0, 1)","(0, 1)","(0, 1)","(0, 1)","(0, 1)","(1, 0)","(1, 0)",True
2183,"(0, 1)","(0, 1)","(0, 1)","(0, 1)","(0, 1)","(1, 0)","(0, 1)",True
2184,"(0, 1)","(0, 1)","(0, 1)","(0, 1)","(0, 1)","(0, 1)","(0, 0)",True
2185,"(0, 1)","(0, 1)","(0, 1)","(0, 1)","(0, 1)","(0, 1)","(1, 0)",False


In [ ]:
# Summarize the scenarios into something meaningful

def format_result(i, outcome):
    match = df.iloc[start + i]
    return f"{match['home']} {outcome[0]} - {match['away']} {outcome[1]}"


options = []
for i in range(int(n / 2)):
    for outcome in outcomes:
        options.append((i, outcome))


def summarize(df):
    scenarios = df.copy()

    # Check if all values are true
    if sum(scenarios["clinched"]) == len(scenarios.index):
        print("Clinched independent of results")
        return

    # Check single game criteria
    for (g1, o1) in options:
        subset = scenarios[scenarios[g1] == o1]
        if sum(subset["clinched"]) == len(subset.index):
            print("Clinched with ", format_result(g1, o1))
            scenarios = scenarios[~(scenarios[g1] == o1)]

    # Check two game criteria
    for ((g1, o1), (g2, o2)) in itertools.combinations(options, 2):
        if g1 == g2:
            continue
        subset = scenarios[(scenarios[g1] == o1) & (scenarios[g2] == o2)]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print(
                "Clinched with ", format_result(g1, o1), " and ", format_result(g2, o2)
            )
            scenarios = scenarios[~((scenarios[g1] == o1) & (scenarios[g2] == o2))]

    # Check three game criteria
    for ((g1, o1), (g2, o2), (g3, o3)) in itertools.combinations(options, 3):
        if len(set([g1, g2, g3])) < 3:
            continue
        subset = scenarios[
            (scenarios[g1] == o1) & (scenarios[g2] == o2) & (scenarios[g3] == o3)
        ]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print(
                "Clinched with ",
                format_result(g1, o1),
                " and ",
                format_result(g2, o2),
                " and ",
                format_result(g3, o3),
            )
            scenarios = scenarios[
                ~((scenarios[g1] == o1) & (scenarios[g2] == o2) & (scenarios[g3] == o3))
            ]

    # Check four game criteria
    for ((g1, o1), (g2, o2), (g3, o3), (g4, o4)) in itertools.combinations(options, 4):
        if len(set([g1, g2, g3, g4])) < 4:
            continue
        subset = scenarios[
            (scenarios[g1] == o1)
            & (scenarios[g2] == o2)
            & (scenarios[g3] == o3)
            & (scenarios[g4] == o4)
        ]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print(
                "Clinched with ",
                format_result(g1, o1),
                " and ",
                format_result(g2, o2),
                " and ",
                format_result(g3, o3),
                " and ",
                format_result(g4, o4),
            )
            scenarios = scenarios[
                ~(
                    (scenarios[g1] == o1)
                    & (scenarios[g2] == o2)
                    & (scenarios[g3] == o3)
                    & (scenarios[g4] == o4)
                )
            ]

    # Check five game criteria
    for ((g1, o1), (g2, o2), (g3, o3), (g4, o4), (g5, o5)) in itertools.combinations(
        options, 5
    ):
        if len(set([g1, g2, g3, g4, g5])) < 5:
            continue
        subset = scenarios[
            (scenarios[g1] == o1)
            & (scenarios[g2] == o2)
            & (scenarios[g3] == o3)
            & (scenarios[g4] == o4)
            & (scenarios[g5] == o5)
        ]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print(
                "Clinched with ",
                format_result(g1, o1),
                " and ",
                format_result(g2, o2),
                " and ",
                format_result(g3, o3),
                " and ",
                format_result(g4, o4),
                " and ",
                format_result(g5, o5),
            )
            scenarios = scenarios[
                ~(
                    (scenarios[g1] == o1)
                    & (scenarios[g2] == o2)
                    & (scenarios[g3] == o3)
                    & (scenarios[g4] == o4)
                    & (scenarios[g5] == o5)
                )
            ]

    # Check six game criteria
    for (
        (g1, o1),
        (g2, o2),
        (g3, o3),
        (g4, o4),
        (g5, o5),
        (g6, o6),
    ) in itertools.combinations(options, 6):
        if len(set([g1, g2, g3, g4, g5, g6])) < 6:
            continue
        subset = scenarios[
            (scenarios[g1] == o1)
            & (scenarios[g2] == o2)
            & (scenarios[g3] == o3)
            & (scenarios[g4] == o4)
            & (scenarios[g5] == o5)
            & (scenarios[g6] == o6)
        ]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print(
                "Clinched with ",
                format_result(g1, o1),
                " and ",
                format_result(g2, o2),
                " and ",
                format_result(g3, o3),
                " and ",
                format_result(g4, o4),
                " and ",
                format_result(g5, o5),
                " and ",
                format_result(g6, o6),
            )
            scenarios = scenarios[
                ~(
                    (scenarios[g1] == o1)
                    & (scenarios[g2] == o2)
                    & (scenarios[g3] == o3)
                    & (scenarios[g4] == o4)
                    & (scenarios[g5] == o5)
                    & (scenarios[g6] == o6)
                )
            ]


summarize(scenarios)

Clinched with  WAS 0 - HOU 0
Clinched with  WAS 1 - HOU 0
Clinched with  NC 0 - BFC 0  and  POR 0 - CHI 0
Clinched with  NC 0 - BFC 0  and  POR 0 - CHI 1
Clinched with  NC 0 - BFC 0  and  LOU 0 - LA 0
Clinched with  NC 0 - BFC 0  and  LOU 1 - LA 0
Clinched with  NC 1 - BFC 0  and  POR 0 - CHI 0
Clinched with  NC 1 - BFC 0  and  POR 0 - CHI 1
Clinched with  NC 1 - BFC 0  and  LOU 0 - LA 0
Clinched with  NC 1 - BFC 0  and  LOU 1 - LA 0
Clinched with  POR 0 - CHI 0  and  RGN 0 - NJY 0
Clinched with  POR 0 - CHI 0  and  RGN 0 - NJY 1
Clinched with  POR 0 - CHI 1  and  RGN 0 - NJY 0
Clinched with  POR 0 - CHI 1  and  RGN 0 - NJY 1
Clinched with  LOU 0 - LA 0  and  RGN 0 - NJY 0
Clinched with  LOU 0 - LA 0  and  RGN 0 - NJY 1
Clinched with  LOU 1 - LA 0  and  RGN 0 - NJY 0
Clinched with  LOU 1 - LA 0  and  RGN 0 - NJY 1
